<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/TP1_M2_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP1

Repositorio de codigo: [GitHub](https://github.com/AgustinBatistelli/programacion_concurrente)

## Parte 1

### Implementación usando C++


Escribo el archivo tp1.cpp con la lógica de mi programa.

In [1]:
%%writefile tp1.cpp
#include <iostream>
#include <string>
#include <sys/wait.h>
#include <unistd.h>

#define HIJOS_DE_A 3
#define HIJOS_DE_B 2
#define HIJOS_DE_C 0
#define HIJOS_DE_D 1
#define HIJOS_DE_E HIJOS_DE_C
#define HIJOS_DE_F HIJOS_DE_E
#define HIJOS_DE_G HIJOS_DE_F

#define PROCESO_B 0
#define PROCESO_C 1
#define PROCESO_E 0

#define TIEMPO_DORMIDO 60
#define INICIO_ITERADOR 0
#define PROCESO_CREADO_CORRECTAMENTE 0
#define PROCESO_SIN_HIJOS 0
#define SALIDA_EXITOSA EXIT_SUCCESS
#define SALIDA_ERRONEA EXIT_FAILURE

bool procesoCreadoCorrectamente(pid_t pid);
void esperarPorMisHijos(int numeroHijos);
void mostrarMiInformacion(char letra, pid_t miPid, pid_t miParentPid);

bool procesoCreadoCorrectamente(pid_t pid)
{
  return pid >= PROCESO_CREADO_CORRECTAMENTE;
}

void esperarPorMisHijos(int numeroHijos)
{
  for (int i = INICIO_ITERADOR; i < numeroHijos; i++)
  {
    wait(NULL);
  }
}

void mostrarMiInformacion(char letra, pid_t miPid, pid_t miParentPid)
{
  std::cout<<"Soy el proceso "<<letra<<". Mi PID es "<<miPid<<" y el PID de mi padre es "<<miParentPid<<std::endl;
}

//Clase IProceso (simula una Interfaz)
class IProceso 
{
  protected:
    pid_t miPid;
    pid_t miPPid;
    char letra;
    int cantidadHijos;

  public:
    IProceso(pid_t pid, pid_t ppid, char caracter, int hijos) : miPid(pid), miPPid(ppid), letra(caracter), cantidadHijos(hijos) {}
    virtual void realizarAccion() = SALIDA_EXITOSA;
};

//Procesos que tienen hijos y deben esperarlos
class ProcesoConHijos : public IProceso 
{
  public:
    ProcesoConHijos(pid_t pid, pid_t ppid, char caracter, int hijos) : IProceso(pid, ppid, caracter, hijos) {}
    void realizarAccion() override 
    {
      mostrarMiInformacion(this->letra,this->miPid,this->miPPid);
      esperarPorMisHijos(this->cantidadHijos);
    }
};

//Procesos que no tienen hijos y deben hacer sleep
class ProcesoSinHijos : public IProceso 
{
  public:
    ProcesoSinHijos(pid_t pid, pid_t ppid, char caracter, int hijos) : IProceso(pid, ppid, caracter, hijos) {}
    void realizarAccion() override 
    {
      mostrarMiInformacion(this->letra,this->miPid,this->miPPid);
      sleep(TIEMPO_DORMIDO);
    }
};

class ProcesoFabrica
{
  public:
    IProceso* determinarTipoProceso(pid_t pid, pid_t ppid, char caracter, int hijos)
    {
      if (hijos > PROCESO_SIN_HIJOS)
      {
        proceso = new ProcesoConHijos(pid,ppid,caracter,hijos);
      }
      else
      {
        proceso = new ProcesoSinHijos(pid,ppid,caracter,hijos);
      }
      return proceso;
    }
  private:
    IProceso* proceso;
};

int main(int argc, char *argv[])
{
  pid_t pidProcesoA, pidProcesoB, pidProcesoD;
  ProcesoFabrica fabricaDeProcesos;

  for (int i = INICIO_ITERADOR; i < HIJOS_DE_A; i++)
  {
    pidProcesoA = fork();
    if (!procesoCreadoCorrectamente(pidProcesoA))
    {
      std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
      return SALIDA_ERRONEA;
    }
    if (!pidProcesoA)
    {
      if (i == PROCESO_B)
      {                
        for (int j = INICIO_ITERADOR; j < HIJOS_DE_B; j++)
        {
          pidProcesoB = fork();
          if (!procesoCreadoCorrectamente(pidProcesoB))
          {
            std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
            return SALIDA_ERRONEA;
          }
          if (!pidProcesoB)
          {
            if (j == PROCESO_E)
            {
              IProceso* procesoE = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'E',HIJOS_DE_E);
              procesoE->realizarAccion();
              return SALIDA_EXITOSA;
            }
            else
            {
              IProceso* procesoF = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'F',HIJOS_DE_F);
              procesoF->realizarAccion();
              return SALIDA_EXITOSA;
            }
          }                
        }
        IProceso* procesoB = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'B',HIJOS_DE_B);
        procesoB->realizarAccion();
        return SALIDA_EXITOSA;
      }
      else
      {
        if (i == PROCESO_C)
        {
          IProceso* procesoC = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'C',HIJOS_DE_C);
          procesoC->realizarAccion();
          return SALIDA_EXITOSA;
        }
        else
        {
          pidProcesoD = fork();
          if (!procesoCreadoCorrectamente(pidProcesoD))
          {
            std::cout<<"Se ha producido un error al crear un nuevo proceso"<<std::endl;
            return SALIDA_ERRONEA;
          }
          if (!pidProcesoD)
          {
            IProceso* procesoG = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'G',HIJOS_DE_G);
            procesoG->realizarAccion();
            return SALIDA_EXITOSA;
          }
          IProceso* procesoD = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'D',HIJOS_DE_D);
          procesoD->realizarAccion();
          return SALIDA_EXITOSA;
        }
      }    
    }
  }
  IProceso* procesoA = fabricaDeProcesos.determinarTipoProceso(getpid(),getppid(),'A',HIJOS_DE_A);
  procesoA->realizarAccion();
  return SALIDA_EXITOSA;
}

Writing tp1.cpp


Listo los archivos para ver que el archivo tp1.cpp esté creado correctamente.

In [2]:
!ls -la

total 24
drwxr-xr-x 1 root root 4096 Apr 12 16:43 .
drwxr-xr-x 1 root root 4096 Apr 12 16:42 ..
drwxr-xr-x 4 root root 4096 Apr 11 13:32 .config
drwxr-xr-x 1 root root 4096 Apr 11 13:33 sample_data
-rw-r--r-- 1 root root 5158 Apr 12 16:43 tp1.cpp


Compilo el archivo tp1.cpp para obtener un archivo a ejecutar. 

In [3]:
!g++ -std=c++11 -o tp1 tp1.cpp

Listo los archivos nuevamente para verificar que tengo el ejecutable del proceso. 

In [4]:
!ls -la

total 44
drwxr-xr-x 1 root root  4096 Apr 12 16:43 .
drwxr-xr-x 1 root root  4096 Apr 12 16:42 ..
drwxr-xr-x 4 root root  4096 Apr 11 13:32 .config
drwxr-xr-x 1 root root  4096 Apr 11 13:33 sample_data
-rwxr-xr-x 1 root root 19112 Apr 12 16:43 tp1
-rw-r--r-- 1 root root  5158 Apr 12 16:43 tp1.cpp


Ejecuto el proceso en segundo plano. 

In [5]:
!nohup ./tp1 1>salidaC 2>/dev/null & 

Obtengo los procesos que se crearon al ejecutar el programa.

In [6]:
!ps -ef | grep tp1

root         366       1  0 16:43 ?        00:00:00 ./tp1
root         367     366  0 16:43 ?        00:00:00 ./tp1
root         368     366  0 16:43 ?        00:00:00 ./tp1
root         369     367  0 16:43 ?        00:00:00 ./tp1
root         370     366  0 16:43 ?        00:00:00 ./tp1
root         371     367  0 16:43 ?        00:00:00 ./tp1
root         372     370  0 16:43 ?        00:00:00 ./tp1
root         389     166  0 16:43 ?        00:00:00 /bin/bash -c ps -ef | grep tp1
root         391     389  0 16:43 ?        00:00:00 grep tp1


Obtengo el árbol de procesos que nace a partir del proceso padre (A)

In [7]:
!pstree -p 366       

tp1(366)─┬─tp1(367)─┬─tp1(369)
         │          └─tp1(371)
         ├─tp1(368)
         └─tp1(370)───tp1(372)


Imprimo la salida de pantalla de cada proceso. 

In [ ]:
!cat salidaC

Soy el proceso A. Mi PID es 416 y el PID de mi padre es 1
Soy el proceso D. Mi PID es 419 y el PID de mi padre es 416
Soy el proceso B. Mi PID es 417 y el PID de mi padre es 416
Soy el proceso E. Mi PID es 421 y el PID de mi padre es 417
Soy el proceso C. Mi PID es 418 y el PID de mi padre es 416
Soy el proceso G. Mi PID es 420 y el PID de mi padre es 419
Soy el proceso F. Mi PID es 422 y el PID de mi padre es 417


### Implementacion usando Python

Escribo el archivo fork.py con la logica del programa

In [8]:
%%writefile fork.py
#!/usr/bin/env python3


import os
import time

def parent():
    
    info("A", os.getpid(), os.getppid())
    hijos_a = ["B", "C", "D"]
    hijos_b = ["E", "F"]
    
    pid_hijos_a = []
    pid_hijos_b = []
    
    for hijo_a in hijos_a:
        pid_a = os.fork()
        
        if pid_a < 0:
            print("Error al crear el nuevo proceso ", hijo_a)
            os._exit(1)
        
        
        if pid_a == 0:
            if hijo_a == "B":
                info(hijo_a, os.getpid(), os.getppid())
                for hijo_b in hijos_b:
                    pid_b = os.fork()
                    if pid_b < 0:
                        print("Error al crear el nuevo proceso ", hijo_b)
                        os.exit(1)
                        
                    if pid_b == 0:
                        info(hijo_b, os.getpid(), os.getppid())
                        time.sleep(100)
                        os._exit(0)
                    else:
                        pid_hijos_b.append(pid_b);
                for pid_hijo_b in pid_hijos_b:
                    os.waitpid(pid_hijo_b,0)
                os._exit(0)
            if hijo_a == "C":
                info(hijo_a, os.getpid(), os.getppid())
                time.sleep(100)
                os._exit(0)
            else:
                info(hijo_a, os.getpid(), os.getppid())
                pid_d = os.fork()
                if pid_d < 0:
                    print("Error al crear el nuevo proceso ", hijo_a)
                    os.exit(1)
                if pid_d == 0:
                    info("G", os.getpid(), os.getppid())
                    time.sleep(100)
                    os._exit(0)
                else:
                    os.wait()
                os._exit(0)
        else:
            pid_hijos_a.append(pid_a)
                
    for pid_hijo_a in pid_hijos_a:
        os.waitpid(pid_hijo_a,0)
        
    os._exit(0)

def info(nombre_proceso, pid, ppid):
    print("Proceso ", nombre_proceso , " con PID: ", pid, " y PID de padre: ", ppid)
    
parent()

Writing fork.py


Ejecutamos el programa en segundo plano

In [9]:
!nohup python fork.py 1>salidaPython 2>/dev/null &

Buscamos con ps el proceso que esta ejecutando nuestro programa

In [10]:
!ps -ef | grep fork.py


root         488       1  1 16:44 ?        00:00:00 python3 fork.py
root         490     488  0 16:44 ?        00:00:00 python3 fork.py
root         491     488  0 16:44 ?        00:00:00 python3 fork.py
root         492     488  0 16:44 ?        00:00:00 python3 fork.py
root         493     490  0 16:44 ?        00:00:00 python3 fork.py
root         494     490  0 16:44 ?        00:00:00 python3 fork.py
root         495     492  0 16:44 ?        00:00:00 python3 fork.py
root         508     166  0 16:44 ?        00:00:00 /bin/bash -c ps -ef | grep fork.py
root         510     508  0 16:44 ?        00:00:00 grep fork.py


Mostramos el árbol de procesos que se generó

In [11]:
!pstree -pc 488               

python3(488)─┬─python3(490)─┬─python3(493)
             │              └─python3(494)
             ├─python3(491)
             └─python3(492)───python3(495)


Mostrar la salida del proceso ejecutandolo nuevamente

In [ ]:
!python fork.py

Proceso  A  con PID:  8070  y PID de padre:  158
Proceso  B  con PID:  8072  y PID de padre:  8070
Proceso  D  con PID:  8074  y PID de padre:  8070
Proceso  F  con PID:  8076  y PID de padre:  8072
Proceso  C  con PID:  8073  y PID de padre:  8070
Proceso  E  con PID:  8075  y PID de padre:  8072
Proceso  G  con PID:  8077  y PID de padre:  8074


### Implementacion usando JAVA

Descargar la libreria gson para java.

In [12]:
!curl https://repo1.maven.org/maven2/com/google/code/gson/gson/2.10.1/gson-2.10.1.jar --output "gson-2.10.1.jar"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  276k  100  276k    0     0  1976k      0 --:--:-- --:--:-- --:--:-- 1976k


Validar la descarga de la libreria

In [13]:
!ls -lash

total 332K
4.0K drwxr-xr-x 1 root root 4.0K Apr 12 16:44 .
4.0K drwxr-xr-x 1 root root 4.0K Apr 12 16:42 ..
4.0K drwxr-xr-x 4 root root 4.0K Apr 11 13:32 .config
4.0K -rw-r--r-- 1 root root 2.1K Apr 12 16:44 fork.py
280K -rw-r--r-- 1 root root 277K Apr 12 16:44 gson-2.10.1.jar
4.0K -rw-r--r-- 1 root root  418 Apr 12 16:43 salidaC
   0 -rw-r--r-- 1 root root    0 Apr 12 16:44 salidaPython
4.0K drwxr-xr-x 1 root root 4.0K Apr 11 13:33 sample_data
 20K -rwxr-xr-x 1 root root  19K Apr 12 16:43 tp1
8.0K -rw-r--r-- 1 root root 5.1K Apr 12 16:43 tp1.cpp


Generar el archivo con datos de entrada.

In [14]:
!echo '{"A":["B","C","D"],"B":["E","F"],"D":["G"]}' > data.json

Generar el archivo con el programa necesario.

In [34]:
%%writefile ProcessStuff.java
import com.google.gson.Gson;
import com.google.gson.reflect.TypeToken;
import java.lang.reflect.Type;
import java.io.StringReader;
import java.io.PrintWriter;
import java.io.FileOutputStream;
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.FileSystems;
import java.nio.file.FileSystem;
import java.util.*;

public class ProcessStuff {
    public static void main(String[] args) {
        String self = "";
        if (args.length == 0) {
          self = "A";
        } else {
          self = String.valueOf(args[0]);
        }
        Gson gson = new Gson();
        Type customType = new TypeToken<HashMap<String, ArrayList<String>>>() {}.getType();
        HashMap<String, ArrayList<String>> dataMap = gson.fromJson(getFileData(DATA_FILE_NAME), customType); 
        
        info(self);

        ArrayList<Process> children = spawnChildren(dataMap, self);
        for(Process child : children) {
            try {
              child.waitFor();
            } catch (InterruptedException e) {
              System.err.println("Se ha producido una interrupción");
            }
        }
        try {
          Thread.sleep(5000);
        } catch (InterruptedException e) {
          System.err.println("Se ha producido una interrupción");
        }
    }

    public static String getFileData(String fileName) {
        String data = "";
        try {
            data = String.join("", Files.readAllLines(FileSystems.getDefault().getPath(fileName)));
        } catch (NullPointerException e) {
            e.printStackTrace();
        } catch (IOException e) {
            System.err.println("No se pudo leer el archivo " + fileName);
            e.printStackTrace();
        }
        return data;
    }

    public synchronized static void info(String self) {
        ProcessHandle selfProcessHandle = ProcessHandle.current();
        try (
            PrintWriter out = new PrintWriter(new FileOutputStream(OUT, true), true);
        ) {
            out.print("Soy el proceso " + self + ". ");
            out.print("Mi PID es: " + selfProcessHandle.pid() + ". ");
            out.print("El PID de mi padre es: " + selfProcessHandle.parent().get().pid() + ".\n");
            out.flush();
        } catch (IOException e) {
            System.err.println("No se pudo abrir el archivo: " + OUT);
            e.printStackTrace();
        } catch (NoSuchElementException e) {
            System.err.println("No tengo padre: " + OUT);
            e.printStackTrace();
        }
    }

    public static ArrayList<Process> spawnChildren(HashMap<String, ArrayList<String>> dataMap, String self) {
        ArrayList<Process> children = new ArrayList<>();
        if (dataMap.keySet().contains(self)) {
            for(String childValue : dataMap.get(self)) {
              try {
                children.add(new ProcessBuilder("java", "-cp", ".:gson-2.10.1.jar:gson-2.10.1.jar:", "ProcessStuff", childValue, "1>salidaJava", "2>errors").start());
              } catch (IOException e) {
                System.err.println("No se pudieron crear mas procesos:  " + childValue);
              }
            }
        }
        return children;
    }

    final static String DATA_FILE_NAME = "data.json";
    final static String OUT = "miSalidaJava";
}

Overwriting ProcessStuff.java


Compilar el programa junto con la libreria gson.

In [35]:
!javac -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff.java

Ejecutar el programa en segundo plano.

In [41]:
!nohup java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff 1>salidaJava 2>errors &  

In [42]:
!ps -ef | grep java

root       13364       1 12 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff
root       13381   13364 14 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff B 1>salidaJava 2>errors
root       13384   13364 14 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff C 1>salidaJava 2>errors
root       13387   13364 14 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff D 1>salidaJava 2>errors
root       13439   13387 23 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff G 1>salidaJava 2>errors
root       13444   13381 23 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff E 1>salidaJava 2>errors
root       13453   13381 21 17:32 ?        00:00:00 java -cp .:gson-2.10.1.jar:gson-2.10.1.jar: ProcessStuff F 1>salidaJava 2>errors
root       13501     166  0 17:32 ?        00:00:00 /bin/bash -c ps -ef | grep java
root     

Mostrar el arbol de procesos generado

In [43]:
!pstree -pcT "$(eval ps -eo pid,ppid,args | awk '$NF ~ /ProcessStuff/ && $2 == 1 {print $1}')"                              

java(13364)─┬─java(13381)─┬─java(13444)
            │             └─java(13453)
            ├─java(13384)
            └─java(13387)───java(13439)


Mostrar resultado del archivo de salida del proceso

In [44]:
!cat miSalidaJava

Soy el proceso A. Mi PID es: 13364. El PID de mi padre es: 1.
Soy el proceso D. Mi PID es: 13387. El PID de mi padre es: 13364.
Soy el proceso C. Mi PID es: 13384. El PID de mi padre es: 13364.
Soy el proceso B. Mi PID es: 13381. El PID de mi padre es: 13364.
Soy el proceso E. Mi PID es: 13444. El PID de mi padre es: 13381.
Soy el proceso G. Mi PID es: 13439. El PID de mi padre es: 13387.
Soy el proceso F. Mi PID es: 13453. El PID de mi padre es: 13381.


Borrar el archivo, en caso de volver a ejecutar.

In [40]:
!rm miSalidaJava

## Parte 2

## Parte 3

## Parte 4